In [1]:
import pandas as pd
from country_cleaner import country_clean
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import widgets, interactive

In [2]:
easd_df = pd.read_csv("Input_Data/educational_attainment_supplementary_data.csv")

easd_df = easd_df[easd_df["Keep or not"] == 1]

cols_to_keep = ["country_name", "series_name", "1985", "1990", "1995", "2000", "2005", "2010"]
easd_df.drop(columns=[col for col in easd_df if col not in cols_to_keep], inplace=True)

easd_df['country_name'] = easd_df['country_name'].apply(lambda x: country_clean(x))


In [3]:
barro_lee_ser = ['Barro-Lee: Average years of total schooling, age 15+, female', 'Barro-Lee: Average years of total schooling, age 15+, total', 'Barro-Lee: Average years of total schooling, age 25+, female', 'Barro-Lee: Average years of total schooling, age 25+, total', 'Barro-Lee: Percentage of population age 15+ with no education', 'Barro-Lee: Percentage of population age 25+ with no education']

barro_lee_df = easd_df[easd_df['series_name'].isin(barro_lee_ser)]
barro_lee_df.dropna(axis=0, thresh=6, inplace=True)


countries = barro_lee_df['country_name'].unique()
categories = ['Average years of total schooling, age 15+', 'Average years of total schooling, age 25+', 'Percent of population with no schooling, age 15+', 'Percent of population with no schooling, age 25+']
category_matcher = {'Average years of total schooling, age 15+': ['Barro-Lee: Average years of total schooling, age 15+, female', 'Barro-Lee: Average years of total schooling, age 15+, total'], 'Average years of total schooling, age 25+':['Barro-Lee: Average years of total schooling, age 25+, female', 'Barro-Lee: Average years of total schooling, age 25+, total'], 'Percent of population with no schooling, age 15+':['Barro-Lee: Percentage of population age 15+ with no education'], 'Percent of population with no schooling, age 25+': ['Barro-Lee: Percentage of population age 25+ with no education']}

In [4]:
country_dropdown = widgets.Dropdown(options=list(countries), description="Country: ")
category_dropdown = widgets.Dropdown(options=categories, description="Category: ")

def plot_barro(country, category):
    df = barro_lee_df.copy()

    #Filter country
    df = df[df['country_name']==country]

    #Filter category
    chosen_categories = category_matcher[category]
    df = df[df['series_name'].isin(chosen_categories)]
    df.reset_index(drop=True, inplace=True)
    df.drop(['country_name'], axis=1, inplace=True)
    df.set_index('series_name', inplace=True)

    plot_df = df.T

    for cat in chosen_categories:
        plt.plot(["1985", "1990", "1995", "2000", "2005", "2010"], cat, data=plot_df, label=cat)
    
    plt.legend()
    plt.show()

    return 

In [5]:
interactive(plot_barro, country=country_dropdown, category=category_dropdown)

interactive(children=(Dropdown(description='Country: ', options=('Afghanistan', 'Albania', 'Algeria', 'Argenti…